In [1]:
import datetime
start = datetime.date(2025,12,1)
end = datetime.date(2025,12,31)

In [2]:
import numpy as np
import pandas as pd
import time
import os.path
import shutil

#path = 'E:\\dtuklaptop\\e\\Users\\Mat\python\\data\\property\\bank-download\\'
#generated_path = 'E:\\dtuklaptop\\e\\Users\\Mat\\python\\data\\property\\generated\\'

path='J://My Drive//NAS//My Documents//Business//Property//Statements//working//python//data//property//bank-download//'
generated_path = 'J://My Drive//NAS//My Documents//Business//Property//Statements//working//python//data//property//generated//'

def backup_file(filename):
    timestr = time.strftime('%Y%m%d-%H%M%S')
    if os.path.isfile(filename):
        shutil.copy2(filename,filename+'.bak_' + timestr)
    return filename

def load_barclays(filename):
    input_file = path + filename
    print('loading: ' + input_file)
    df=pd.read_csv(input_file,names=['Number','Date','Account','Amount','Subcategory','Memo','Memo2'],skiprows=1, index_col=1, parse_dates=True, dayfirst=True)
    df=df.replace('\t','', regex=True)
    # Need to do this because some memo columns contain commas which confuses read_csv
    df['Memo2'] = df['Memo2'].fillna('')
    df['Memo'] = df['Memo'].astype(str) + df['Memo2'].astype(str)
    df.drop(['Memo2','Number'], axis=1, inplace=True)
    #df.set_index('Date')
    #df = categorise(df)
    return df

def load_starling(filename):
    input_file = path + filename
    print('loading: ' + input_file)
    df=pd.read_csv(input_file, index_col=0, parse_dates=True, dayfirst=True)
    df['Account'] = '60-83-71 00558156'
    df['Counter Party'] = df['Counter Party'].fillna('')
    df['Reference'] = df['Reference'].fillna('')
    df['Notes'] = df['Notes'].fillna('')
    df['Memo'] = df['Counter Party'] + ' ' + df['Reference']  + ' ' + df['Notes']
    df.rename(columns = {'Amount (GBP)':'Amount','Spending Category':'Subcategory'}, inplace = True)
    df.drop(['Counter Party','Reference','Notes','Balance (GBP)'], axis=1, inplace=True)
    df = df[['Account','Amount','Subcategory','Memo']]
    return df

### Process Barclays and Starling data into standard format

- input files are:
    - Barclays files as BC_6045_MONYYYY.csv, BC_3072_MONYYYY.csv and BC_4040_MONYYYY.csv
    - RSA Capital data from Starling account as is (StarlingStatement_YYYY-MM.csv)
- processes above files into single bank file for each month in a standard format
- output files are 'MMMYYYY.csv' in generated folder

!!!!!!! Unicode problem - need to open Starling file and save as UTF-8

In [3]:
from dateutil.rrule import rrule, MONTHLY

dates = [dt for dt in rrule(MONTHLY, dtstart=start, until=end)]
for date in dates:
    dateStr= date.strftime("%b").upper() + date.strftime("%Y")
    starlingDateStr= date.strftime("%Y") + '-' + date.strftime("%m")
    dfMt=load_barclays('BC_6045_'+ dateStr +'.csv')
    dfIv=load_barclays('BC_3072_'+ dateStr +'.csv')
    df40=load_barclays('BC_4040_'+ dateStr +'.csv')
    dfRsa=load_starling('StarlingStatement_'+starlingDateStr+'.csv')
    dfAll=pd.concat([dfMt,dfIv,df40,dfRsa]).sort_index()
    
    # Save file
    file=backup_file(generated_path + dateStr + '.csv')
    print('Creating: ' + file)
    dfAll.to_csv(file)

loading: J://My Drive//NAS//My Documents//Business//Property//Statements//working//python//data//property//bank-download//BC_6045_DEC2025.csv
loading: J://My Drive//NAS//My Documents//Business//Property//Statements//working//python//data//property//bank-download//BC_3072_DEC2025.csv
loading: J://My Drive//NAS//My Documents//Business//Property//Statements//working//python//data//property//bank-download//BC_4040_DEC2025.csv
loading: J://My Drive//NAS//My Documents//Business//Property//Statements//working//python//data//property//bank-download//StarlingStatement_2025-12.csv
Creating: J://My Drive//NAS//My Documents//Business//Property//Statements//working//python//data//property//generated//DEC2025.csv
